In [1]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pinecone_env = os.environ["PINECONE_ENV"]

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings

In [3]:
# pwd

In [4]:
# import os

# # Specify the path
# path = "data"

# # Use os.listdir() to list all entries in the specified path
# entries = os.listdir(path)

# # Print each entry in the specified path
# for entry in entries:
#     print(entry)

In [5]:
# import json
# from pathlib import Path
# from pprint import pprint


# file_path=r'data/dummy.json'
# data = json.loads(Path(file_path).read_text())

In [6]:
# pprint(data)

In [7]:
# # from langchain_community.document_loaders import JSONLoader

# # loader = JSONLoader(
# #     file_path = r"data\dummy.json",
# #     jq_schema='.[]',)
# from langchain_community.document_loaders import TextLoader

# loader = TextLoader(r"data\dummy.json")
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()

In [8]:
# import pinecone
# from pinecone import Pinecone, ServerlessSpec, PodSpec

# pc = Pinecone(
#     api_key=pinecone_api_key
# )

# index_name = "dimension-demo"

# # First, check if our index already exists. If it doesn't, we create it
# if index_name not in pc.list_indexes():
#     # we create a new index
#     pc.create_index(name=index_name, metric="cosine", dimension=1536, spec = PodSpec(environment=pinecone_env))
# # The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
# docsearch = pc.from_documents(docs, embeddings, index_name=index_name)

# # if you already have an index, you can load it like this
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)

# query = "What is john doing"
# docs = docsearch.similarity_search(query)

In [9]:
# from pinecone import Pinecone, PodSpec

# pc = Pinecone(api_key=pinecone_api_key)

# pc.create_index(
#   name="pod-index",
#   dimension=1536,
#   metric="cosine",
#   spec=PodSpec(
#     environment="gcp-starter"
#   )
# )

In [10]:
# # First, check if our index already exists. If it doesn't, we create it
# # if index_name not in pc.list_indexes():
# #     # we create a new index
# #     pc.create_index(name=index_name, metric="cosine", dimension=1536, spec = PodSpec(environment=pinecone_env))
# # The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
# docsearch = pc.from_documents(docs, embeddings, index_name="pod-index")

# # if you already have an index, you can load it like this
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)

# query = "What is john doing"
# docs = docsearch.similarity_search(query)

In [11]:
from pinecone import Pinecone, PodSpec
import time

pc = Pinecone(api_key=pinecone_api_key,
             environment=pinecone_env)

existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

index_name = "pod-index"

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
          name= index_name,
          dimension=1536,
          metric="cosine",
          spec=PodSpec(
            environment="gcp-starter"
          )
        )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [13]:
import os

from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
import requests
load_dotenv()

embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_existing_index(index_name=index_name,
                                           embedding=embeddings)

retriever = vectorstore.as_retriever()

In [14]:
def fetch_url(x):
    loader = TextLoader(r"data\dummy.txt")
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)   
    return {"context": docs, "question": x["question"]}

# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | RunnableLambda(fetch_url)  # Add this line
    | prompt
    | model
    | StrOutputParser()
)

C:\Users\ACER\Desktop\work_trial\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [15]:
print(chain.invoke("are there any duplicate tasks, tasks which have similar description"))

Created a chunk of size 9418, which is longer than the specified 1000


Based on the provided context, there are tasks with similar descriptions that suggest they might be duplicates or closely related tasks. Here are the tasks with similar descriptions:

1. Task with "Id": "8bd5fcea-3c56-4771-ac41-54321fe98734" and Task with "Id": "9f8d7c6e-ab7d-4e2c-8c34-56789f0ed3f2" both have the title "Database Optimization" and similar descriptions related to optimizing the database schema and reviewing indexes for effectiveness.

2. Task with "Id": "6a8dcef8-3dae-4567-ada4-1a2b3c4d5e6f" and Task with "Id": "7b8e9d2c-69d1-48a7-b3f1-45678ef9abcd" both have the title "Bug Fix on Homepage" with descriptions indicating issues on the homepage, one related to layout on mobile devices and the other to a JavaScript error in the console.

3. Task with "Id": "4e9b7ecd-60ae-4b35-ae8c-22393abcd119", Task with "Id": "a49b8e7f-c1d2-460e-999d-54e345678bcf", Task with "Id": "2a3b4c5d-6e7f-8a9b-cdef-1234567890ab", Task with "Id": "q1w2e3r4-t5y6-u7i8-o9p0-a1s2d3f4g5h6", and Task with 

In [45]:
# rag_chain_with_source = RunnableParallel(
#     {"context": retriever, "question": RunnablePassthrough()}
# ).assign(answer=chain)

# print(rag_chain_with_source.invoke("what is john doing"))

In [40]:
# # upsert the data in 
# from langchain_community.document_loaders import TextLoader

# loader = TextLoader(r"data\dummy.txt")
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()

# # docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

# # if you already have an index, you can load it like this
# # docsearch = Pinecone.from_existing_index(index_name, embeddings)

# # for batch in dataset.iter_documents(batch_size=100):
#     # index.upsert(batch)

In [41]:
# index.upsert(docs, embeddings)

In [42]:
# query = "What did the president say about Ketanji Brown Jackson"
# docs = docsearch.similarity_search(query)

In [43]:
# print(docs[0].page_content)

In [44]:
# # More text can embedded and upserted to an existing Pinecone index using the add_texts function

# index = pinecone.Index("langchain-demo")
# vectorstore = Pinecone(index, embeddings.embed_query, "text")

# vectorstore.add_texts("More text!")